In [53]:
import gym
import numpy as np
import tensorflow as tf
from collections import namedtuple

In [6]:
xavier = tf.contrib.layers.xavier_initializer
print(xavier)

<function xavier_initializer at 0x7f2f46aeaf28>


In [16]:
env = gym.make('BreakoutDeterministic-v4')
n_actions = env.action_space.n

In [21]:
states_input = tf.placeholder(shape=(None, 105, 80, 4), dtype=tf.float32)
actions_input = tf.placeholder(shape=(n_actions,), dtype=tf.int8)


conv1 = tf.layers.conv2d(frames_input, 16, (8, 8), (4, 4),
                             activation=tf.nn.relu,
                             kernel_initializer=xavier(),
                             bias_initializer=xavier())
conv2 = tf.layers.conv2d(conv1, 32, (4, 4), (2, 2),
                             activation=tf.nn.relu,
                             kernel_initializer=xavier(),
                             bias_initializer=xavier())
hidden = tf.layers.dense(tf.layers.flatten(conv2), 256,
                             activation=tf.nn.relu,
                             kernel_initializer=xavier(),
                             bias_initializer=xavier())

In [18]:
print(frames_input)
print(actions_input)
print(conv1)
print(conv2)
print(hidden)

Tensor("Placeholder_2:0", shape=(?, 105, 80, 4), dtype=float32)
Tensor("Placeholder_3:0", shape=(4,), dtype=int8)
Tensor("conv2d/Relu:0", shape=(?, 25, 19, 16), dtype=float32)
Tensor("conv2d_2/Relu:0", shape=(?, 11, 8, 32), dtype=float32)
Tensor("dense/Relu:0", shape=(?, 256), dtype=float32)


In [32]:
frames_input = tf.placeholder(shape=(None, 105, 80, 4), dtype=tf.uint8)
tf.to_float(frames_input) / 255.

<tf.Tensor 'truediv_6:0' shape=(?, 105, 80, 4) dtype=float32>

In [107]:
def preprocess_img(img):
    """
    Images are converted from RGB to grayscale and downsampled by a factor of
    2. Deepmind actually used a final 84x84 image by cropping since their GPU
    wanted a square input for convolutions. We do not preprocess, rather we
    store as uint8 for the sake of memory.
    :param img: Atari image (210, 160, 3)
    :return: Grayscale downsample version (105, 80)
    """
    return np.mean(img[::2, ::2], axis=2).astype(np.uint8)

In [124]:
def normalize(states):
    return tf.to_float(states) / 255.

In [110]:
raw_img, reward, done, info = env.step(0)
img = preprocess_img(raw_img)
img.shape

(105, 80)

In [111]:
state = np.stack((img, img, img, img), axis=2)
state.shape

(105, 80, 4)

In [112]:
np.mean(state)

40.097142857142856

In [122]:
new_state = np.concatenate((state[:, :, :3], np.reshape(img+4, (105, 80, 1))), axis=2)
new_state.shape

(105, 80, 4)

In [123]:
np.mean(new_state)

41.097142857142856

In [129]:
batch_states = [state, new_state]
normalize(np.array(state))

<tf.Tensor 'truediv_9:0' shape=(105, 80, 4) dtype=float32>

In [51]:
[.9999**(10**i) for i in (0, 1, 2, 3, 4, 5, 6)]

[0.9999,
 0.9990004498800211,
 0.9900493386913719,
 0.9048328935585562,
 0.36786104643297046,
 4.537723395901116e-05,
 3.7015207857933866e-44]

In [54]:
Experience = namedtuple(
    'Experience',
    ['state', 'action', 'reward', 'next_state', 'done'])

In [130]:
1 - 1e-5

0.99999

In [64]:
img =

(210, 160, 3)

In [80]:
np_state = np.array(state)
np_state.shape

(4,)

In [81]:
state[0].shape

(210, 160, 3)

In [79]:
ll = [np.mean(img[::2, ::2], axis=2).astype(np.uint8) for img in state]

TypeError: 'float' object is not subscriptable

In [84]:
type(state[0])

numpy.ndarray

In [85]:
[type(img) for img in state]

[numpy.ndarray, float, bool, dict]